In [1]:
import numpy as np
import time

## Generate the Data

We first generate a random dataset with number of features (m = 10) and number of instances (n = 100)
We also generate a random label vector y \in {-1,1}

In [2]:
n = 100
m = 10

X = np.random.rand(n,m)
wgen = np.random.rand(m)
y = np.dot(X,wgen) + np.random.normal(0, 0.1, n)
w = np.random.rand(m)

## A Simple naive Implementation of the Least Squares

Below is a simple naive implementation of Least Square Loss. We directly plug in the formula with a simple for loop!

In [3]:
def LeastSquaresNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    f = 0
    g = 0
    for i in range(len(X)):
        featureweightProd = np.dot(X[i],w)
        f = f + (featureweightProd - y[i])*(featureweightProd - y[i])
        g = g + 2*(featureweightProd - y[i])*X[i]
    f = f + 0.5*lam*np.sum(w*w)
    g = g + lam*w.reshape(1,-1)
    return [f, g]     

In [4]:
start = time.time()
[f,g] = LeastSquaresNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 0.0012369155883789062
Function value = 18.11430323106446
Printing Gradient:
[[-26.40587054 -10.08174756 -20.06086287 -26.54846283 -24.77238833
  -19.75775787 -28.39635462 -27.87900945 -32.93241229 -30.65521977]]


## For Loop in Python == Slow Code

Great, we have a working code now. But while this code might be correct, is this going to be fast? We have a For loop in python which is clearly an issue!

First let us see how slow the code is! Let us increase n to 10000000 and m to 1000, which are somewhat more realistic (though still far from real world).

In [5]:
n = 1000000
m = 100

X = np.random.rand(n,m)
wgen = np.random.rand(m)
y = np.dot(X,wgen) + np.random.normal(0, 0.1, n)
w = np.random.rand(m)

start = time.time()
[f,g] = LeastSquaresNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))

Time Taken = 3.6550180912017822
Function value = 2979067.713990548


## Speeding up the code!

With n = 10000000, it takes around 2 minutes to run a single function evaluation!

Lets now vectorize the code below.

In [6]:
def LeastSquares(w, X, y, lam):
    # Computes the cost function for all the training samples
    m = X.shape[0]
    Xw = np.matmul(X,w)
    Xwy = (Xw - y).reshape(-1,1)
    f = np.dot(Xwy.T,Xwy) + 0.5*lam*np.sum(w*w)
    g = 2*np.dot(X.T,Xwy)  + lam*w.reshape(-1,1)
    return [f, g]

In [7]:
start = time.time()
[f,g] = LeastSquares(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print(np.shape(g))

Time Taken = 0.1029510498046875
Function value = [[2979067.71399057]]
(100, 1)


## Checking gradient implementations!

So far so good! But how do we verify if our gradient implementation is correct?
We can test out our loss function analytically, but what if we make a mistake in computing the gradient? We can numerically compute the gradient to ensure it is correct.

In [8]:
def LeastSquaresFun(w, X, y, lam):
    # Computes the cost function for all the training samples
    m = X.shape[0]
    Xw = np.matmul(X,w)
    Xwy = (Xw - y).reshape(-1,1)
    f = np.dot(Xwy.T,Xwy) + 0.5*lam*np.sum(w*w)
    return f

def numericalGrad(funObj, w,epsilon):
    m = len(w)
    grad = np.zeros(m)
    for i in range(m):
        wp = np.copy(w)
        wn = np.copy(w)
        wp[i] = w[i] + epsilon
        wn[i] = w[i] - epsilon
        grad[i] = (funObj(wp) - funObj(wn))/(2*epsilon)
    return grad

In [9]:
n = 100
m = 10

X = np.random.rand(n,m)
wgen = np.random.rand(m)
y = np.dot(X,wgen) + np.random.normal(0, 0.1, n)
w = np.random.rand(m)

funObj = lambda w: LeastSquaresFun(w,X,y,1)
[f,g] = LeastSquares(w,X,y,1)
gn = numericalGrad(funObj, w, 1e-10)
fn = funObj(w)
print(f)
print(fn)
print(gn)
print(g)

[[29.21435641]]
[[29.21435641]]
[32.52422331 29.83098213 40.37449486 45.67565881 30.93648004 52.74085169
 38.37445917 41.6134327  38.7920096  46.16868665]
[[32.52420457]
 [29.8309785 ]
 [40.37451531]
 [45.67562758]
 [30.93648274]
 [52.74079572]
 [38.3744738 ]
 [41.61338888]
 [38.79201958]
 [46.16868019]]
